# Inicio Rápido

Esta sección muestra a grandes rasgos la (Application Progrsamming Interface) API para tareas comunes en aprendizaje automático. Consulta los enlaces en cada sección para profundizar al respecto.

## Trabajando con datos

PyTorch tiene dos [primitivas para trabajar con datos](https://pytorch.org/docs/stable/data.html):
``torch.utils.data.DataLoader`` y ``torch.utils.data.Dataset``.
``Dataset`` almacena las muestras y sus etiquetas correspondientes, y ``DataLoader`` envuelve un iterable alrededor
del ``Dataset``.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch ofrece bibliotecas específicas de dominio como [TorchVision](https://pytorch.org/vision/stable/index.html) y [TorchAudio](https://pytorch.org/audio/stable/index.html), las cuales incluyen conjuntos de datos para tareas de visión por computadora o manipulación de audio respectivamente. Para este tutorial, usaremos un conjunto de datos de TorchVision.

El módulo ``torchvision.datasets`` contiene objetos ``Dataset`` para muchos sets de visión del mundo real como
CIFAR, COCO ([lista completa aquí](https://pytorch.org/vision/stable/datasets.html)). En este tutorial, usamos
el conjunto de datos FashionMNIST. Cada ``Dataset`` de TorchVision incluye dos argumentos: ``transform`` y
``target_transform`` para modificar las muestras y etiquetas respectivamente.

In [ ]:
# Descargar datos de entrenamiento de conjuntos de datos abiertos.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Descargar datos de prueba de conjuntos de datos abiertos.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Se pasa el ``Dataset`` como un argumento al objeto/método ``DataLoader``. Esto "envuelve" el dataset en un iterador (iterable) que soporta la generación automática de lotes (batches) así como muestreo (sampling), mezclado (shuffling) y la carga de datos multiproceso.
En el siguiente ejemplo se define un tamaño de lote de 64, con lo que cada elemento del iterador que conforma el dataloader regresara un lote de 64 características y etiquetas.

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Lee más sobre [cargar datos en PyTorch](data_tutorial.html).

---

## Creando Modelos

Para definir una red neuronal en PyTorch, creamos una clase que hereda
de [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html). Definimos las capas de la red
en la función ``__init__`` y especificamos cómo los datos pasarán a través de la red en la función ``forward``. Para acelerar
las operaciones en la red neuronal, la movemos al [acelerador](https://pytorch.org/docs/stable/torch.html#accelerators)
como CUDA, MPS, MTIA, o XPU. Si el acelerador actual está disponible, lo usaremos. De lo contrario, usamos la CPU.

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Usando dispositivo {device}")

In [ ]:
# Definir el modelo
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Lee más sobre [construir redes neuronales en PyTorch](buildmodel_tutorial.html).

---

## Optimizando los Parámetros del Modelo

Para entrenar un modelo, necesitamos una [función de pérdida](https://pytorch.org/docs/stable/nn.html#loss-functions)
y un [optimizador](https://pytorch.org/docs/stable/optim.html).

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

En un único ciclo de entrenamiento, el modelo hace predicciones sobre el conjunto de datos de entrenamiento (alimentado en lotes), y
retropropaga el error de predicción para ajustar los parámetros del modelo.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Calcular error de predicción
        pred = model(X)
        loss = loss_fn(pred, y)

        # Retropropagación
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"pérdida: {loss:>7f}  [{current:>5d}/{size:>5d}]")

También verificamos el rendimiento del modelo contra el conjunto de datos de prueba para asegurar que está aprendiendo.

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Error de Prueba: \n Precisión: {(100*correct):>0.1f}%, Pérdida promedio: {test_loss:>8f} \n")

El proceso de entrenamiento se lleva a cabo a lo largo de varias iteraciones (*épocas*). Durante cada época, el modelo aprende
parámetros para hacer mejores predicciones. Imprimimos la precisión y la pérdida del modelo en cada época; buscando que la
precisión aumente y la pérdida disminuya con cada época.

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Época {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("¡Listo!")

Lee más sobre [Entrenar tu modelo](optimization_tutorial.html).

---

## Guardar Modelos

Una forma común de guardar un modelo es serializar el diccionario de estado interno (que contiene los parámetros del modelo).

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Estado del Modelo PyTorch guardado en model.pth")

## Cargar Modelos

El proceso para cargar un modelo incluye recrear la estructura del modelo y cargar
el diccionario de estado en él.

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

Este modelo ahora puede ser usado para hacer predicciones.

In [ ]:
classes = [
    "Camiseta/top",
    "Pantalón",
    "Suéter",
    "Vestido",
    "Abrigo",
    "Sandalia",
    "Camisa",
    "Zapatilla",
    "Bolsa",
    "Bota",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicción: "{predicted}", Real: "{actual}"')

Lee más sobre [Guardar y Cargar tu modelo](saveloadrun_tutorial.html).

### Representación gráfica de la inferencia del modelo

In [ ]:
# Mostrar las primeras 4 imágenes de los conjuntos de datos de entrenamiento y prueba
import matplotlib.pyplot as plt


# Crear subgráficas para los datos de entrenamiento
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
fig.suptitle('Conjunto de Datos Fashion-MNIST - Primeras 4 Imágenes', fontsize=16)

# Mostrar las primeras 4 imágenes de entrenamiento
for i in range(4):
    image, label = training_data[i]
    axes[0, i].imshow(image.squeeze(), cmap='gray')
    axes[0, i].set_title(f'Entrena: {classes[label]}')
    axes[0, i].axis('off')

# Mostrar las primeras 4 imágenes de prueba
for i in range(4):
    image, label = test_data[i]
    axes[1, i].imshow(image.squeeze(), cmap='gray')
    axes[1, i].set_title(f'Prueba: {classes[label]}')
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

# Imprimir detalles de las imágenes
print(f"Tamaño del conjunto de entrenamiento: {len(training_data)}")
print(f"Tamaño del conjunto de prueba: {len(test_data)}")
print(f"Forma de la imagen: {training_data[0][0].shape}")
print(f"Número de clases: {len(classes)}")